# Notebook 5: Weather predictions for: Łódź, Gdańsk, Szczecin, Rzeszów, Warszawa and Kraków
Each csv file for different cities contains data collected in 2022 or 2022/23. Predictions will be executed for 2022 and MAE (mean absolute error) will be calculated for each prediction.

In the notebook comparison of predictions made by LSTM model and XGBoost algorithm will be conducted.

Predictions will be executed for different Polish cities. The main goal is to check the influence on predictions (made in different cities) of learning LSTM models on data from only Łódź and learning XGBoost algorithm on data from Warszawa, Wrocław, Szczecin, Rzeszów.

XGBoost algorithms will make a predictions for data in each city separately and then will save them.

LSTM models will be updated by data from Łódź every seven days and then predictions for the next seven days will be executed.

Important note: first execute XGB_model.ipynb and LSTM_model_different_windows_sizes_and_50epochs.ipynb notebooks for models creation.

## All necessary libraries imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.metrics import mean_absolute_error

Imports from helpful_functions.python script located in root/notebooks folder.

In [2]:
from helpful_functions import  min_max_denormalization, transform_data

## Files to load

In [3]:
# minimum and maximum values for denormalization read
min_and_max = pd.read_csv("generated_models/lstm_models/min_and_max")
min = min_and_max['min']
max = min_and_max['max']

In [4]:
# XGBoost biases
xgb_bias_df = pd.read_csv("generated_models/xgb_models/biases_xgboost")
xgb_bias_df.drop('Unnamed: 0',axis='columns', inplace=True)

xgb_bias = []

for i in range(3):
    xgb_bias.append(xgb_bias_df.iloc[i].values)

In [5]:
xgb_bias

[array([ 0.23349158, -0.21502533, -0.00570393]),
 array([ 0.24179471, -0.25555926, -0.02740839]),
 array([ 0.18760138, -0.33614829, -0.02982774])]

In [6]:
# models load
# LSTM models
model_lstm = []
model_lstm.append(pickle.load(open("generated_models/lstm_models/lstm_1.pkl", "rb", -1)))
model_lstm.append(pickle.load(open("generated_models/lstm_models/lstm_2.pkl", "rb", -1)))
model_lstm.append(pickle.load(open("generated_models/lstm_models/lstm_3.pkl", "rb", -1)))

# XGBoost models
model_xgb = []
model_xgb.append(pickle.load(open("generated_models/xgb_models/xgb_1.pkl", "rb", -1)))
model_xgb.append(pickle.load(open("generated_models/xgb_models/xgb_2.pkl", "rb", -1)))
model_xgb.append(pickle.load(open("generated_models/xgb_models/xgb_3.pkl", "rb", -1)))

In [13]:
# Read the CSV file for with data from  Łódź -  data necessary for making LSTM models updates
Lodz_data = pd.read_csv("all_data/data_distance_from_Lodz/Lodz_2022.csv")
Lodz_lstm = Lodz_data[["relh", "skph", "temp"]]

## Predictions for each city

In [8]:
# longtitude and latitude of cities necessary for XGBoost models
long_lat = {"Lodz_2022_lon":19,"Lodz_2022_lat":51,
    "Gdansk_2022_2023_lon":18.64,"Gdansk_2022_2023_lat":54.35, "Krakow_2022_2023_lon":19.9450,"Krakow_2022_2023_lat":50.0647, "Rzeszow_2022_2023_lon": 22, "Rzeszow_2022_2023_lat":50.04  ,
      "Szczecin_2022_2023_lon":14.55, "Szczecin_2022_2023_lat":53.42, "Warszawa_2022_2023_lon":21.01, "Warszawa_2022_2023_lat":52.22
}

# names of folders with cities data to load
all_cities = ["Lodz_2022", "Gdansk_2022_2023", "Krakow_2022_2023", "Rzeszow_2022_2023", "Szczecin_2022_2023", "Warszawa_2022_2023"]

# dictionaries for MAE saving for predictions evaluation
MAE_lstm_all = {}
MAE_xgb_all = {}

Predictions and MAE by XGBoost models.

In [9]:

########################### XGB #############################

for city in all_cities:
    # log
    print(city)

    # name of file to read data from
    fileName = "all_data/data_distance_from_Lodz/" + city + ".csv"
    all_data = pd.read_csv(fileName)
    # predictions for 2022 only
    all_data = all_data[all_data['year']==2022]

    # data for xGBoost model #####################
    all_data.drop('Unnamed: 0',axis='columns', inplace=True)
    all_data_x = all_data.copy()
    all_data_x.rename(columns ={'temp':'tmpc', 'skph':'sped'}, inplace=True)
    all_data_x['hour'] = pd.to_datetime(all_data['time']).dt.hour
    all_data_x.drop('time',axis='columns', inplace=True)

    # different lon and lat
    lon = long_lat[city+"_lon"]
    lat = long_lat[city+"_lat"]
    all_data_x.insert(loc=0, column="lat", value=lat) 
    all_data_x.insert(loc=0, column="lon", value=lon) 
    all_data_x=all_data_x[all_data_x['minutes']==0]
    all_data_x.drop('minutes',axis='columns', inplace=True)

    ########################### XGB #############################
    MAE_humid_xgb_temp = []
    MAE_wind_xgb_temp = []
    MAE_temp_xgb_temp = []

    for hour in range(1,4): # cause predicitons for next 3 hours
        # make predictionss
        X = all_data_x[:-hour]
        y = all_data_x[hour:]
        y_pred_xgb = model_xgb[hour-1].predict(X)
        y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=[ "lon", "lat",  "tmpc",  "relh" ,"sped" ,"day","month", "year", "hour"])

        MAE_humid_xgb_temp.append(mean_absolute_error(y_pred_xgb[["relh"]] + xgb_bias[hour-1][0],y[["relh"]]))
        MAE_wind_xgb_temp.append(mean_absolute_error(y_pred_xgb[["sped"]] + xgb_bias[hour-1][1],y[["sped"]]))
        MAE_temp_xgb_temp.append(mean_absolute_error(y_pred_xgb[["tmpc"]] + xgb_bias[hour-1][2],y[["tmpc"]]))

    # save MAE for each place
    MAE_xgb_all[city] = [MAE_humid_xgb_temp, MAE_wind_xgb_temp, MAE_temp_xgb_temp]     


Lodz_2022
Gdansk_2022_2023
Krakow_2022_2023
Rzeszow_2022_2023
Szczecin_2022_2023
Warszawa_2022_2023


Predictions and MAE by LSTM models.

In [14]:

######################### LSTM ###############################

# paraeters needed for updates
how_many_updates = 55 # because one year has ~50 weeks and 320 timestamps it is 6.66 days 
size_of_timestamps_in_updating_set = 320
window_size = 12
epochs = 5

# converting data frames to  numpy arrays
Lodz_lstm = Lodz_lstm.to_numpy()

for update in range(how_many_updates):
    
    for hour in range(3):
        # every model update with data from Łódź
        # data for update
        data_patch_update = Lodz_lstm[update * size_of_timestamps_in_updating_set : (update+1) * size_of_timestamps_in_updating_set + window_size + (hour*2)+1]
        X, y = transform_data(data_patch_update, max, min, timestamps_count = (hour*2)+1, is_update = True)
        len = y.shape[0]
        len = y.shape[0] - (y.shape[0] % (32))
        X = X[:len]
        y = y[:len] 
        # model update
        model_lstm[hour].reset_states()
        model_lstm[hour].fit(X, y,  epochs=epochs, shuffle=False , verbose = 0, batch_size=32)

    for city in all_cities:
        fileName = "all_data/data_distance_from_Lodz/" + city + ".csv"
        all_data = pd.read_csv(fileName)

        # data for LSTM model #####################
        all_data_lstm = all_data[["relh", "skph", "temp"]]
        all_data_lstm=all_data_lstm.to_numpy()

        ######################### LSTM ###############################

        MAE_humid_lstm_temp = []
        MAE_wind_lstm_temp = []
        MAE_temp_lstm_temp = []

        for hour in range(3): # cause predicitons for next 3 hours
            # choose proper patch set
            data_patch_test = all_data_lstm[(update+1) * size_of_timestamps_in_updating_set : (update+2) * size_of_timestamps_in_updating_set + window_size + (hour*2)+1]
            X_test, y_test = transform_data(data_patch_test, max, min, timestamps_count = (hour*2)+1, is_update = True)
            
            len = y_test.shape[0]
            len = y_test.shape[0] - (y_test.shape[0] % (32))
            X_test = X_test[:len]
            y_test = y_test[:len] 

            # make predictions
            model_lstm[hour].reset_states()
            predictions = model_lstm[hour].predict(X_test, verbose = 0, batch_size=32)

            pred = []
            actual =[]

            # denormalization of data
            weather_components_size = y.shape[1]
            for i in range(weather_components_size):
                denormalized = min_max_denormalization(predictions[:,i], max[i], min[i])
                pred.append(denormalized)
                actual.append(min_max_denormalization(y_test[:,i], max[i], min[i]))

            # MAE for one patch set
            MAE_humid_lstm_temp.append(mean_absolute_error(actual[0], pred[0]))
            MAE_wind_lstm_temp.append(mean_absolute_error(actual[1], pred[1]))
            MAE_temp_lstm_temp.append(mean_absolute_error(actual[2], pred[2]))

        # save data for each place - sum previously collected MAE with the new from the last patch set of data
        if city in MAE_lstm_all:
            MAE_lstm_all[city] = [np.array(MAE_humid_lstm_temp)/how_many_updates + MAE_lstm_all[city][0], np.array(MAE_wind_lstm_temp)/how_many_updates + MAE_lstm_all[city][1], np.array(MAE_temp_lstm_temp)/how_many_updates + MAE_lstm_all[city][2]]  
        else:
            MAE_lstm_all[city] = [np.array(MAE_humid_lstm_temp)/how_many_updates, np.array(MAE_wind_lstm_temp)/how_many_updates, np.array(MAE_temp_lstm_temp)/how_many_updates]    

InvalidArgumentError: Graph execution error:

Detected at node while/add defined at (most recent call last):
  File "c:\Users\AWeg\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\AWeg\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\AWeg\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\AWeg\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\AWeg\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\AWeg\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Users\AWeg\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 600, in run_forever

  File "c:\Users\AWeg\AppData\Local\Programs\Python\Python310\lib\asyncio\base_ev

## Results

LSTM models and XGBoost algorithms predictions comparison in dataframes for each weather condition: humidity, speed of wind and temperature.

The second and third columns shows MAE calculated for Łódź city made by LSTM and XGBoost.

Row number 0: MAE of predictions for the next hour.

Row number 1: MAE of predictions for the second hour.

Row number 2: MAE of predictions for the third hour.

In [ ]:
# MAE for relative humidity in %
data_h = pd.DataFrame()
data_h['hour'] = [1,2,3]
for city in all_cities:
    name_lstm = city[:3] + "_lstm"
    data_h[name_lstm] = MAE_lstm_all[city][0]

    name_xgb = city[:3] + "_xgb"
    data_h[name_xgb] = MAE_xgb_all[city][0]

# display dataframe showing humidity MAE
data_h

,hour,Lod_lstm,Lod_xgb,Gda_lstm,Gda_xgb,Kra_lstm,Kra_xgb,Rze_lstm,Rze_xgb,Szc_lstm,Szc_xgb,War_lstm,War_xgb
0,1,3.996115,4.785962,4.160826,4.519108,4.094247,4.854247,4.032097,4.508461,4.074480,4.695234,4.009824,4.612573
1,2,5.777544,6.480153,5.984442,6.199690,5.750878,6.489112,5.894290,6.132640,5.873312,6.293921,5.812468,6.309258
2,3,7.502553,7.903206,7.602911,7.371013,7.309163,7.854646,7.650798,7.473511,7.453557,7.571683,7.463089,7.885851


In [ ]:
# MAE for speed of wind in km/h
data_w = pd.DataFrame()
data_w['hour'] = [1,2,3]
for city in all_cities:
    name_lstm = city[:3] + "_lstm"
    data_w[name_lstm] = MAE_lstm_all[city][1]

    name_xgb = city[:3] + "_xgb"
    data_w[name_xgb] = MAE_xgb_all[city][1]

# display dataframe showing speed of wind MAE
data_w

,hour,Lod_lstm,Lod_xgb,Gda_lstm,Gda_xgb,Kra_lstm,Kra_xgb,Rze_lstm,Rze_xgb,Szc_lstm,Szc_xgb,War_lstm,War_xgb
0,1,2.718939,3.641866,2.901391,3.656817,2.749438,3.577358,2.976287,3.565937,2.726439,3.420197,2.688027,3.333078
1,2,3.491978,4.260552,3.814785,4.354838,3.449759,4.413943,3.699023,4.296856,3.491437,4.141746,3.393419,4.000617
2,3,4.088203,4.948526,4.496444,4.928411,3.989480,5.178344,4.291325,4.840964,4.015923,4.701326,3.936044,4.465582


In [ ]:
# MAE for temperature in °C
data_t = pd.DataFrame()
data_t['hour'] = [1,2,3]
for city in all_cities:
    name_lstm = city[:3] + "_lstm"
    data_t[name_lstm] = MAE_lstm_all[city][2]

    name_xgb = city[:3] + "_xgb"
    data_t[name_xgb] = MAE_xgb_all[city][2]

# display dataframe showing temperature MAE
data_t

,hour,Lod_lstm,Lod_xgb,Gda_lstm,Gda_xgb,Kra_lstm,Kra_xgb,Rze_lstm,Rze_xgb,Szc_lstm,Szc_xgb,War_lstm,War_xgb
0,1,0.763685,0.870979,0.700873,0.788922,0.758884,0.900873,0.741050,0.801298,0.742661,0.813717,0.727400,0.844084
1,2,1.211119,1.327925,1.121274,1.160593,1.199206,1.327586,1.194973,1.175691,1.205620,1.194055,1.140105,1.206139
2,3,1.680032,1.705307,1.545418,1.565809,1.661822,1.738262,1.671535,1.544754,1.657363,1.543835,1.572779,1.603993


LSTM models predictions comparison in dataframes for each weather condition: humidity, speed of wind and temperature.

The second column shows MAE calculated for Łódź city. The other columns shows respectively MAE calculated for Gdańsk, Kraków, Rzeszów, Szczecin and Warszawa.

Row number 0: MAE of predictions for the next hour.

Row number 1: MAE of predictions for the second hour.

Row number 2: MAE of predictions for the third hour.

In [ ]:
# humidity LSTM only
# MAE for relative humidity in %
data_h_lstm = pd.DataFrame()
data_h_lstm['hour'] = [1,2,3]
for city in all_cities:
    name_lstm = city[:3] + "_lstm"
    data_h_lstm[name_lstm] = MAE_lstm_all[city][0]

data_h_lstm

,hour,Lod_lstm,Gda_lstm,Kra_lstm,Rze_lstm,Szc_lstm,War_lstm
0,1,3.996115,4.160826,4.094247,4.032097,4.074480,4.009824
1,2,5.777544,5.984442,5.750878,5.894290,5.873312,5.812468
2,3,7.502553,7.602911,7.309163,7.650798,7.453557,7.463089


In [ ]:
# speed of wind LSTM only
# MAE for speed of wind in km/h
data_w_lstm = pd.DataFrame()
data_w_lstm['hour'] = [1,2,3]
for city in all_cities:
    name_lstm = city[:3] + "_lstm"
    data_w_lstm[name_lstm] = MAE_lstm_all[city][1]

data_w_lstm

,hour,Lod_lstm,Gda_lstm,Kra_lstm,Rze_lstm,Szc_lstm,War_lstm
0,1,2.718939,2.901391,2.749438,2.976287,2.726439,2.688027
1,2,3.491978,3.814785,3.449759,3.699023,3.491437,3.393419
2,3,4.088203,4.496444,3.989480,4.291325,4.015923,3.936044


In [ ]:
# temperature LSTM only
# MAE for temperature in °C
data_t_lstm = pd.DataFrame()
data_t_lstm['hour'] = [1,2,3]
for city in all_cities:
    name_lstm = city[:3] + "_lstm"
    data_t_lstm[name_lstm] = MAE_lstm_all[city][2]

data_t_lstm

,hour,Lod_lstm,Gda_lstm,Kra_lstm,Rze_lstm,Szc_lstm,War_lstm
0,1,0.763685,0.700873,0.758884,0.741050,0.742661,0.727400
1,2,1.211119,1.121274,1.199206,1.194973,1.205620,1.140105
2,3,1.680032,1.545418,1.661822,1.671535,1.657363,1.572779


### Conclusion:
Even the LSTM models have been learned on only historic data from Łódź city and XGBoost algorithms have been learned on data from Warszawa, Wrocław, Szczecin, Rzeszów - there is no huge impact on predictions for different polish cities, at least on this tested in this notebook.